In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *


In [0]:
dbutils.widgets.text("incremental_flag", "0")


In [0]:
incremental_flag = dbutils.widgets.get("incremental_flag")

In [0]:
%sql

select * from parquet.`abfss://slvr@adlsgen5485.dfs.core.windows.net/carsales`

##Creating Dimension Table

In [0]:
df_src = spark.sql("SELECT DISTINCT Model_ID, model_category FROM parquet.`abfss://slvr@adlsgen5485.dfs.core.windows.net/carsales`")
df_src.display()

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.dim_model"):
    df_sink = spark.sql("select dim_model_key,model_id,model_category from cars_catalog.gold.dim_model")
else:
    df_sink = spark.sql("select 0 dim_model_key,model_id,model_category from parquet.`abfss://slvr@adlsgen5485.dfs.core.windows.net/carsales` where 1=0")

In [0]:
df_sink.display()

##Filtering New Records and Old Records

In [0]:
df_filter = df_src.join(df_sink, df_src['Model_ID'] == df_sink['Model_ID'], "left").select(df_src['Model_ID'],df_src['model_category'],df_sink['dim_model_key'])


In [0]:
df_filter.display()

In [0]:
df_filter_old = df_filter.filter(df_filter['dim_model_key'].isNotNull())
df_filter_new = df_filter.filter(df_filter['dim_model_key'].isNull()).select('Model_ID','model_category')



In [0]:
if (incremental_flag == "0"):
    max_value = 1

else:
    if spark.catalog.tableExists("cars_catalog.gold.dim_model"):
        max_value_df = spark.sql("select max(dim_model_key) from cars_catalog.gold.dim_model")
        max_value = max_value_df.collect()[0][0]+1
    



In [0]:
df_filter_new = df_filter_new.withColumn('dim_model_key',max_value + monotonically_increasing_id())
df_filter_new.display()

In [0]:
df_final = df_filter_new.union(df_filter_old)
df_final.display()

##SCD Type 1(Upsert)

In [0]:

#Incremental Run
if spark.catalog.tableExists("cars_catalog.gold.dim_model"):
  delta_table = DeltaTable.forPath(spark, "abfss://gold@adlsgen5485.dfs.core.windows.net/dim_model")
  delta_table.alias("t").merge(df_final.alias("s"), "t.dim_model_key = s.dim_model_key") \
      .whenMatchedUpdateAll() \
      .whenNotMatchedInsertAll() \
      .execute()
#Initial Run      
else:
   df_final.write.format("delta") \
     .mode("overwrite").option("path","abfss://gold@adlsgen5485.dfs.core.windows.net/dim_model") \
     .saveAsTable("cars_catalog.gold.dim_model")

In [0]:
%sql

select * from cars_catalog.gold.dim_model;
